<a href="https://colab.research.google.com/github/mjolnika/japanese-russian-translationese/blob/main/Japanese_Russian_Translationese_prepare_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Compute additional features

Addendum to code by Kunilovskaya & Lapshinova-Koltunski (https://github.com/kunilovskaya/translationese45)

## directs

regular expressions from hselingapi direct speech (https://github.com/hseling/hseling-api-direct-speech/blob/master/hseling_api_direct_speech/csv_files/speech.csv)

In [ ]:
#LEFT_CONTEXT9speech9RIGHT_CONTEXT
spdict = []
speech = r'''(?<!:)\n\s*?9[—–-]{1,2}.+?[\.\?!]*?9$
[\.\?!]\s*9[…][А-Я][а-яё ,—–-]*:\n\s*?[—–-]{1,2}.+?[\.\?!]*?9$
^9[А-Я][а-яёА-Я ,—–-]*:\n\s*?[—–-]{1,2}.+?[\.\?!]*?9$
[\.\?!>]\s*9[«“"][А-Я]?[а-яёА-Я ,—–-]*[:;] [«“"][А-Я][ёа-я ,\?!\.…—–-]+[”»"], [ёа-я ,]+[”»"]9[\.\?!,]*?
[\.\?!>]\s*9[«“"][А-Я]?[а-яёА-Я ,—–-]*[:;] [«“"][А-Я][ёа-яА-Я ,\?!\.…;—–-]+[”»"] — [ёа-я ,]+[”»"]9[\.\?!,]*?
\s*9"[А-Я]?[а-яёА-Я ,—–-]*[:;] [«“"][А-Я][ёа-яА-Я ,\?!\.…;—–-]+[”»"] [ёа-я ,]+"9[\.\?!,]*?
[^:][ \n]9[«“"][А-Я][а-яёА-Я \.\?!—–-]*[”»"], [а-яёА-Я ]*.(?<!\w\.\w.)(?<![А-Я][а-я]\.)(?<=\.|\?)9\s
[^:][ \n]9[«“"][А-Я][а-яА-Яё ,!\?\.]*[”»"], [а-яА-Яё ]*, [«“"][а-яА-Яё ,!\?\.]*[”»"]9/s
[^:][ \n]9[«“"][А-Я][А-Яа-яё, !\?\.—–-]*[”»"]9 —–-{1,2}
[\.\?!]\s9[А-Я][а-яё ,—–-]*: [«“"][а-яА-Яё ,—–-]*[”»"]9\.
 — 9[«“"][а-яёА-Я\?!\. ,—–-]*[”»"]9\s[^а-яА-Яё]
 9[«“"][а-яА-Яё ,.\?!—–-]*[”»"], [—–-] [а-яА-Я ,—–-]*\.9\s
\s*9"[А-Я][А-Яа-яё ,—–-]+[:;] [«“"][а-яА-Я ;]+[”»"]"9[!\.\?,]*
\s*9"[А-Я][-ёа-яА-Я ,;]+: [«“"][А-Я][-а-я ,]+[”»"]"9, [—–-] [<«]
\s*9[«“"][А-Я][А-Яа-я ё\?!,]+[”»"]9 [—–-]{1,2} [«<]
 [—–-] 9"[«“"][А-Я][А-Яа-я ,;-]+[”»"]."9\s
\s9"[«“"][А-Я][А-Яа-я ,;!\?\.—–-]+[”»"], [а-яА-Я ]+, «[А-Яа-я ,;!\?—–-]+[”»"]"9 [—–-]{1,2} [«<]'''.splitlines()

for line in speech:
    
    left, s, right = re.split('9', line)
    spdict.append([left, s, right])

In [ ]:
def countdirect(path):
    try:
        with open(path, encoding='utf-8') as f:
            text =  f.read()
            
    except:
        with open(path) as f:
            text =  f.read()
            
    text = re.sub('(\r\n)+', '\n', text)
    text = re.sub('\n+', '\n', text)

    cnting = 0
    for nn, el in enumerate(spdict):
    
        regex = ''.join(el)
        found = re.findall(regex, text, flags=re.M)
        
        
        cnting += len(found)
        
    return cnting

Код для выделения причастий

In [ ]:
def countparticiples(parsed, filename, corp, default=0):
    total = 0
    meaninsent = []
    status = 'participles'
    for par in parsed:
        insentence = 0
        
        for n, p in enumerate(par):
            if p['deprel'] == 'amod':
                if p['lemma'].endswith('ть') or p['lemma'].endswith('ти'):
                    total += 1
                    insentence += 1
                    
            if p['deprel'] == 'acl' and p['upos'] == 'VERB':
                if p['feats'].get('VerbForm', 0):
                    
                    if p['feats']['VerbForm'] == 'Part':
                        if p['lemma'].endswith('ть') or p['lemma'].endswith('ти'):
                            total += 1
                            insentence += 1
        if insentence:
            meaninsent.append(insentence)
            
    return total, round(total/len(parsed), 4),  round(mean(meaninsent), 4)

# CUT

Функция для подсчета непрямой речи; выделенные объекты показались нерепрезентативными

In [ ]:
def countindirect(file):
    
    with open(file, encoding='utf-8') as f:
        conn = f.read()
    parsed = parse(conn)
    
    counting = 0
    for par in parsed:
        for n, p in enumerate(par):
            if p['lemma'] in verblist:
                if  p['feats']:
                    if p['feats'].get('VerbForm', 0):
                        if p['feats']['VerbForm'] != 'Inf':
                            if  p['feats'].get('Tense', 0):
                                if p['feats']['Tense'] == 'Past':
                                    if n + 2<len(par):
                                        if par[n+1]['lemma'] == ',' and par[n+2]['lemma'] == 'что' and par[0]['upos'] != 'PUNCT':
                                            
                                            counting += 1
    return counting
                        

Подсчитать субъекты транзитивных глаголов и их одушевленность

In [ ]:
def counttrans(parsed, filename, corp, default=0):
    c = defaultdict(int)
    status = 'trans'
    for par in parsed:
        root = None
        
        
        for n, p in enumerate(par):
            if p['deprel'] == 'root':
                if p['feats']:
                    if p['feats'].get('VerbForm', 0):
                        if p['feats']['VerbForm'] == 'Fin':
                            root = p['id']
                    
        
        if root:
            
            for n, p in enumerate(par):
                
                if p['head'] == root and p['deprel'] == 'obj':
            
            
                    for n, p in enumerate(par):
                        if p['head'] == root and p['deprel'] == 'nsubj':
                            c[p['lemma']] += 1
                            
                            if p.get('feats',0):
                                if p['feats'].get('Animacy', 0):
                                    animacy = p['feats']['Animacy']
                                else:
                                    animacy = p['upos']
                            else:
                                animacy = p['upos']
                                
                            if p['lemma'] not in animacydict:
                                animacydict[p['lemma']] = animacy
                    
    
    year = metadict[filename]
    
    
    if default == 0:
    
        with open(f'{corp}_{status}.tsv', 'a+', encoding='utf-8') as f:
            for k, v in Counter(c).most_common():
                anim = animacydict[k]
                f.write(f'{filename}\t{year}\t{k}\t{v}\t{anim}\n')
                
    else:
        with open(f'{corp}_{status}_{default}.tsv', 'a+', encoding='utf-8') as f:
            for k, v in Counter(c).most_common():
                if v > default:
                    anim = animacydict[k]
                    f.write(f'{filename}\t{year}\t{k}\t{v}\t{anim}\n'

Субъекты пассивных предложений, их одушевленность

In [ ]:
def countpassives(parsed, filename, corp, default=0):
    c = defaultdict(int)
    status = 'passives'
    for par in parsed:
        root = None
        
        
        for n, p in enumerate(par):
            if p['deprel'] == 'root':
                if p['feats']:
                    if p['feats'].get('Voice', 0):
                        if p['feats']['Voice'] == 'Pass':
                            root = p['id']
                
        if root:
            for n, p in enumerate(par):
                if p['head'] == root and 'nsubj'  in p['deprel']:
                    c[p['lemma']] += 1
                    if p.get('feats',0):
                        if p['feats'].get('Animacy', 0):
                            animacy = p['feats']['Animacy']
                        else:
                            animacy = p['upos']
                    else:
                        animacy = p['upos']
                        
                    if p['lemma'] not in animacy:
                        animacydict[p['lemma']] = animacy
                    
    
    year = metadict[filename]
    
    
    if default == 0:
    
        with open(f'{corp}_{status}.tsv', 'a+', encoding='utf-8') as f:
            for k, v in Counter(c).most_common():
                anim = animacydict[k]
                f.write(f'{filename}\t{year}\t{k}\t{v}\t{anim}\n')
                
    else:
        with open(f'{corp}_{status}_{default}.tsv', 'a+', encoding='utf-8') as f:
        
            for k, v in Counter(c).most_common():
                if v > default:
                    anim = animacydict[k]
                    f.write(f'{filename}\t{year}\t{k}\t{v}\t{anim}\n')
        

Предложения без субъекта (по факту находились выражения типа "Будете чай?")

In [ ]:
def countprodrop(parsed, filename, corp, default=0):
    c = defaultdict(int)
    status = 'prodrop'
    for par in parsed:
        root = None
        foundsubj = 0
        
        
        for n, p in enumerate(par):
            if p['deprel'] == 'root':
                if p['feats']:
                    if p['feats'].get('VerbForm', 0):
                        if p['feats']['VerbForm'] == 'Fin':
                            root = p['id']
            
        for n, p in enumerate(par):
                
            if p['head'] == root:
                if p['deprel'] == 'nsubj' or p['deprel'] == 'nsubj:pass':
                    foundsubj = 1
                    
        if foundsubj == 0:
            print(par)
        else:
            print('XXX')